# 1. Library Import

In [364]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# 소수점 둘째자리까지 표시하도록 설정 (원하는 자릿수로 변경 가능)
pd.options.display.float_format = '{:.2f}'.format  


# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

## Optuna tunning for XGB
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import optuna

## Optuna visulization
import plotly.express as px
import plotly.graph_objects as go
import plotly

# 열의 개수를 출력할 때 모두 표시하도록 설정
pd.set_option('display.max_columns', None)


# 2. Data Loading

In [390]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '../data/train2.csv'
test_path  = '../data/test2.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [391]:
train1 = train.copy()
test1 = test.copy()

In [392]:
subway_bus = pd.read_csv('../data/subway_bus.csv')

In [393]:
subway_bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8955 entries, 0 to 8954
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   지번주소      8955 non-null   object 
 1   유동승객수     8955 non-null   int64  
 2   역까지_거리    8955 non-null   float64
 3   좌표X뉴      8955 non-null   float64
 4   좌표Y뉴      8955 non-null   float64
 5   1차역세권     8955 non-null   int64  
 6   2차역세권     8955 non-null   int64  
 7   X좌표       8955 non-null   float64
 8   Y좌표       8955 non-null   float64
 9   정류장까지_거리  8955 non-null   float64
dtypes: float64(6), int64(3), object(1)
memory usage: 699.7+ KB


In [394]:
subway_bus = subway_bus.drop(['좌표X뉴', '좌표Y뉴'], axis = 1)

In [395]:
subway_bus.rename(columns={'지번주소':'FULL_ADRES','유동승객수':'FLOATING_POPULATION','역까지_거리':'SUBWAY_DIST',
                      '1차역세권':'1STSUBAREA','2차역세권':'2NDSUBAREA','정류장까지_거리':'BUS_DIST','X좌표':'X_CODE','Y좌표':'Y_CODE'
                      },inplace=True)

In [396]:
subway_bus = subway_bus.drop(['FLOATING_POPULATION', 'SUBWAY_DIST','1STSUBAREA','2NDSUBAREA','BUS_DIST'], axis = 1)
train1 = train1.drop(['X_CODE','Y_CODE'], axis = 1)
test1 = test1.drop(['X_CODE','Y_CODE'], axis = 1)

In [398]:
train1 = pd.merge(train1,subway_bus,how='left',on='FULL_ADRES')
test1 = pd.merge(test1,subway_bus,how='left',on='FULL_ADRES')

# 3. Calculate Bridge to Apt distance using Haversine Formula

하버사인 공식을 사용하여 한강에 있는 대교들과 아파트 사이의 거리를 구해서 새로운 Feature를 만들었습니다.  
해당 feature는 27개의 대교와 모든 아파트 사이의 거리를 구하고 아파트 마다 가장 짧은 거리를 'BRIDGE_DIST'에 입력될 수 있도록 코드를 짜서 처리하였습니다.  

In [405]:
# data 읽어오기 
bridge = pd.read_csv('../data/bridge.csv')

In [406]:
bridge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   대교      27 non-null     object 
 1   좌표X     27 non-null     float64
 2   좌표Y     27 non-null     float64
dtypes: float64(2), object(1)
memory usage: 776.0+ bytes


In [407]:
bridge1 = bridge.copy()

In [408]:
train2 = train1[['X_CODE','Y_CODE','FULL_ADRES']]
test2 = test1[['X_CODE','Y_CODE','FULL_ADRES']]

In [409]:
# 8943개
train2 = train2.drop_duplicates('FULL_ADRES')
test2 = test2.drop_duplicates('FULL_ADRES')

In [410]:
# 공통된 column을 기준으로 train2와 test2 합치기
data = pd.concat([train2, test2], ignore_index=True)

In [411]:
data = data.drop_duplicates('FULL_ADRES')

In [412]:
veryhard = pd.merge(data,bridge1, how='cross')

In [413]:
veryhard = veryhard.drop(columns=['대교'])

In [414]:
# 하버사인 거리
lat1 = np.deg2rad(veryhard['X_CODE'])
lat2 = np.deg2rad(veryhard['좌표X'])
lon1 = np.deg2rad(veryhard['Y_CODE'])
lon2 = np.deg2rad(veryhard['좌표Y'])
dlat = lat2 - lat1
dlon = lon1 - lon2
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
c = 2 * np.arcsin(np.sqrt(a))

In [415]:
# 미터단위 거리 구하기
veryhard['BRIDGE_DIST'] = c * 6371 * 1000

In [416]:
min_value = veryhard.loc[veryhard.groupby('FULL_ADRES')['BRIDGE_DIST'].idxmin()][['X_CODE','Y_CODE','좌표X','좌표Y','BRIDGE_DIST','FULL_ADRES']]

In [417]:
min_value = min_value[['BRIDGE_DIST','FULL_ADRES']]

In [418]:
min_value.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8955 entries, 168984 to 168426
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BRIDGE_DIST  8955 non-null   float64
 1   FULL_ADRES   8955 non-null   object 
dtypes: float64(1), object(1)
memory usage: 209.9+ KB


In [2]:
# 한강 대교와 아파트 사이의 거리 구한 값을 저장하기
 
min_value.to_csv('../data/아파트별 한강대교까지 거리.csv')

In [420]:
# Data merge

train3 = pd.merge(train1, min_value, how = 'left', on = 'FULL_ADRES')
test3 = pd.merge(test1, min_value, how = 'left', on = 'FULL_ADRES')

# 4. Merge 전세가율 

In [425]:
lease_rate = pd.read_csv('../data/lease_rate.csv',encoding='cp949')

In [426]:
lease_rate.rename(columns={'구':'gu','계약년월':'CONTR_YEAR_MONTH',
                           '전세가율':'LEASE_RATE'},inplace=True)

In [427]:
lease_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CONTR_YEAR_MONTH  3150 non-null   int64  
 1   LEASE_RATE        3150 non-null   float64
 2   gu                3150 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 74.0+ KB


In [428]:
# train2['gu']가 같고, train2['contract_year']가 lease_rate에 있는 연도인 경우 merge
train3 = pd.merge(train3, lease_rate, how='left', on=['gu', 'CONTR_YEAR_MONTH'])
test3 = pd.merge(test3, lease_rate, how='left', on=['gu', 'CONTR_YEAR_MONTH'])

# 5. Merge interest_rate / 금리

In [429]:
interest_rate = pd.read_csv('../data/interest_rate.csv',encoding='cp949')

In [430]:
interest_rate.rename(columns={'contract':'CONTR'},inplace=True)

### 5-1. Making 'CONTR' Feature / 계약 년 월 일

In [431]:
# contract_day가 NaN이거나 소수점이 있는 경우를 고려하여 처리
test3['CONTR_DAY'] = test3['CONTR_DAY'].fillna(0).astype(str).str.replace(r'\.0$', '').str.zfill(2)
train3['CONTR_DAY'] = train3['CONTR_DAY'].fillna(0).astype(str).str.replace(r'\.0$', '').str.zfill(2)

# contract 열 생성
test3['CONTR'] = test3['CONTR_YEAR_MONTH'].astype(int).astype(str) + test3['CONTR_DAY']
train3['CONTR'] = train3['CONTR_YEAR_MONTH'].astype(int).astype(str) + train3['CONTR_DAY']

In [432]:
test3['CONTR'] = test3['CONTR'].astype(int)
train3['CONTR'] = train3['CONTR'].astype(int)

In [433]:
train3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1118822 entries, 0 to 1118821
Data columns (total 46 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   ADRES                   1118822 non-null  object 
 1   LOT_NO                  1118822 non-null  object 
 2   APT_NM                  1116696 non-null  object 
 3   EXCL_AREA_SQM           1118822 non-null  float64
 4   CONTR_YEAR_MONTH        1118822 non-null  int64  
 5   CONTR_DAY               1118822 non-null  object 
 6   FLOOR                   1118822 non-null  int64  
 7   BUILD_YEAR              1118822 non-null  int64  
 8   ADRES_DORO              1118822 non-null  object 
 9   CANCEL_REASON_DATE      5983 non-null     float64
 10  TRADE_TYPE              32371 non-null    object 
 11  AGNCY_LOCATION          29241 non-null    object 
 12  CODEAPTNM               248131 non-null   object 
 13  HSHLDR_TY               249259 non-null   object 
 14  CR

### 5-2. 계약년월일 별 금리 데이터 붙이기

In [434]:
train3.loc[(train3['CONTR'] >= 20060810)&(train3['CONTR'] < 20070712), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20060810, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20070712)&(train3['CONTR'] < 20070809), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20070712, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20070809)&(train3['CONTR'] < 20080807), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20070809, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20080807)&(train3['CONTR'] < 20081009), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20080807, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20081009)&(train3['CONTR'] < 20081027), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20081009, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20081027)&(train3['CONTR'] < 20081107), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20081027, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20081107)&(train3['CONTR'] < 20081211), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20081107, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20081211)&(train3['CONTR'] < 20090109), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20081211, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20090109)&(train3['CONTR'] < 20090212), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20090109, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20090212)&(train3['CONTR'] < 20100709), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20090212, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20100709)&(train3['CONTR'] < 20101116), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20100709, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20101116)&(train3['CONTR'] < 20110113), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20101116, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20110113)&(train3['CONTR'] < 20110310), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20110113, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20110310)&(train3['CONTR'] < 20110610), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20110310, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20110610)&(train3['CONTR'] < 20120712), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20110610, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20120712)&(train3['CONTR'] < 20121011), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20120712, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20121011)&(train3['CONTR'] < 20130509), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20121011, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20130509)&(train3['CONTR'] < 20140814), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20130509, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20140814)&(train3['CONTR'] < 20141015), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20140814, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20141015)&(train3['CONTR'] < 20150312), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20141015, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20150312)&(train3['CONTR'] < 20150611), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20150312, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20150611)&(train3['CONTR'] < 20160609), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20150611, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20160609)&(train3['CONTR'] < 20171130), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20160609, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20171130)&(train3['CONTR'] < 20181130), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20171130, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20181130)&(train3['CONTR'] < 20190718), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20181130, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20190718)&(train3['CONTR'] < 20191016), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20190718, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20191016)&(train3['CONTR'] < 20200317), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20191016, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20200317)&(train3['CONTR'] < 20200528), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20200317, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20200528)&(train3['CONTR'] < 20210826), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20200528, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20210826)&(train3['CONTR'] < 20211125), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20210826, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20211125)&(train3['CONTR'] < 20220114), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20211125, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20220114)&(train3['CONTR'] < 20220414), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20220114, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20220414)&(train3['CONTR'] < 20220526), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20220414, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20220526)&(train3['CONTR'] < 20220713), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20220526, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20220713)&(train3['CONTR'] < 20220825), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20220713, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20220825)&(train3['CONTR'] < 20221012), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20220825, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20221012)&(train3['CONTR'] < 20221124), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20221012, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20221124)&(train3['CONTR'] < 20230113), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20221124, 'interest_rate'].values[0]
train3.loc[(train3['CONTR'] >= 20230113), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20230113, 'interest_rate'].values[0]


In [435]:
test3.loc[(test3['CONTR'] >= 20060810)&(test3['CONTR'] < 20070712), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20060810, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20070712)&(test3['CONTR'] < 20070809), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20070712, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20070809)&(test3['CONTR'] < 20080807), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20070809, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20080807)&(test3['CONTR'] < 20081009), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20080807, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20081009)&(test3['CONTR'] < 20081027), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20081009, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20081027)&(test3['CONTR'] < 20081107), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20081027, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20081107)&(test3['CONTR'] < 20081211), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20081107, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20081211)&(test3['CONTR'] < 20090109), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20081211, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20090109)&(test3['CONTR'] < 20090212), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20090109, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20090212)&(test3['CONTR'] < 20100709), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20090212, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20100709)&(test3['CONTR'] < 20101116), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20100709, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20101116)&(test3['CONTR'] < 20110113), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20101116, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20110113)&(test3['CONTR'] < 20110310), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20110113, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20110310)&(test3['CONTR'] < 20110610), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20110310, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20110610)&(test3['CONTR'] < 20120712), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20110610, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20120712)&(test3['CONTR'] < 20121011), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20120712, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20121011)&(test3['CONTR'] < 20130509), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20121011, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20130509)&(test3['CONTR'] < 20140814), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20130509, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20140814)&(test3['CONTR'] < 20141015), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20140814, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20141015)&(test3['CONTR'] < 20150312), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20141015, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20150312)&(test3['CONTR'] < 20150611), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20150312, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20150611)&(test3['CONTR'] < 20160609), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20150611, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20160609)&(test3['CONTR'] < 20171130), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20160609, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20171130)&(test3['CONTR'] < 20181130), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20171130, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20181130)&(test3['CONTR'] < 20190718), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20181130, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20190718)&(test3['CONTR'] < 20191016), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20190718, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20191016)&(test3['CONTR'] < 20200317), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20191016, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20200317)&(test3['CONTR'] < 20200528), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20200317, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20200528)&(test3['CONTR'] < 20210826), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20200528, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20210826)&(test3['CONTR'] < 20211125), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20210826, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20211125)&(test3['CONTR'] < 20220114), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20211125, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20220114)&(test3['CONTR'] < 20220414), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20220114, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20220414)&(test3['CONTR'] < 20220526), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20220414, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20220526)&(test3['CONTR'] < 20220713), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20220526, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20220713)&(test3['CONTR'] < 20220825), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20220713, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20220825)&(test3['CONTR'] < 20221012), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20220825, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20221012)&(test3['CONTR'] < 20221124), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20221012, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20221124)&(test3['CONTR'] < 20230113), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20221124, 'interest_rate'].values[0]
test3.loc[(test3['CONTR'] >= 20230113), 'interest_rate'] = interest_rate.loc[interest_rate['CONTR'] == 20230113, 'interest_rate'].values[0]


In [436]:
# train3[(train3['CONTR'] >= before) & (train3['CONTR'] < contract)]
# len(train3['interest_rate'].unique().tolist())
len(interest_rate['interest_rate'].unique().tolist())

19

# 6. Merge GDP

In [437]:
gdp = pd.read_csv('../data/gdp.csv',encoding='cp949')

In [438]:
gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CONTR_YEAR_MONTH  68 non-null     int64  
 1   real_gdp          68 non-null     float64
 2   nominal_gdp       68 non-null     object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ KB


In [439]:
gdp['CONTR_YEAR_MONTH'].unique()

array([200612, 200703, 200706, 200709, 200712, 200803, 200806, 200809,
       200812, 200903, 200906, 200909, 200912, 201003, 201006, 201009,
       201012, 201103, 201106, 201109, 201112, 201203, 201206, 201209,
       201212, 201303, 201306, 201309, 201312, 201403, 201406, 201409,
       201412, 201503, 201506, 201509, 201512, 201603, 201606, 201609,
       201612, 201703, 201706, 201709, 201712, 201803, 201806, 201809,
       201812, 201903, 201906, 201909, 201912, 202003, 202006, 202009,
       202012, 202103, 202106, 202109, 202112, 202203, 202206, 202209,
       202212, 202303, 202306, 202309])

### 6-1. 일자에 맞는 GDP 값 merge

In [440]:
# 'contract_year' 값을 정수로 변환
train3['CONTR_YEAR_MONTH'] = train3['CONTR_YEAR_MONTH'].astype(int)

contract_years = sorted([200612, 200703, 200706, 200709, 200712, 200803, 200806, 200809,
       200812, 200903, 200906, 200909, 200912, 201003, 201006, 201009,
       201012, 201103, 201106, 201109, 201112, 201203, 201206, 201209,
       201212, 201303, 201306, 201309, 201312, 201403, 201406, 201409,
       201412, 201503, 201506, 201509, 201512, 201603, 201606, 201609,
       201612, 201703, 201706, 201709, 201712, 201803, 201806, 201809,
       201812, 201903, 201906, 201909, 201912, 202003, 202006, 202009,
       202012, 202103, 202106, 202109, 202112, 202203, 202206, 202209,
       202212, 202303, 202306, 202309])

before = contract_years[0]
# 조건에 따라 'gdp' 값 대입
for contract_year in contract_years[1:]:
    condition = (train3['CONTR_YEAR_MONTH'] > before) & (train3['CONTR_YEAR_MONTH'] <= contract_year)
    gdp_values = gdp[gdp['CONTR_YEAR_MONTH'] == contract_year][['real_gdp','nominal_gdp']].values
    train3.loc[condition, ['real_gdp', 'nominal_gdp']] = gdp_values
    before = contract_year
# 조건에 따라 'gdp' 값 대입
for contract_year in contract_years[1:]:
    condition = (test3['CONTR_YEAR_MONTH'] > before) & (test3['CONTR_YEAR_MONTH'] <= contract_year)
    gdp_values = gdp[gdp['CONTR_YEAR_MONTH'] == contract_year][['real_gdp','nominal_gdp']].values
    test3.loc[condition, ['real_gdp', 'nominal_gdp']] = gdp_values
    before = contract_year

In [441]:
train3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1118822 entries, 0 to 1118821
Data columns (total 49 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   ADRES                   1118822 non-null  object 
 1   LOT_NO                  1118822 non-null  object 
 2   APT_NM                  1116696 non-null  object 
 3   EXCL_AREA_SQM           1118822 non-null  float64
 4   CONTR_YEAR_MONTH        1118822 non-null  int64  
 5   CONTR_DAY               1118822 non-null  object 
 6   FLOOR                   1118822 non-null  int64  
 7   BUILD_YEAR              1118822 non-null  int64  
 8   ADRES_DORO              1118822 non-null  object 
 9   CANCEL_REASON_DATE      5983 non-null     float64
 10  TRADE_TYPE              32371 non-null    object 
 11  AGNCY_LOCATION          29241 non-null    object 
 12  CODEAPTNM               248131 non-null   object 
 13  HSHLDR_TY               249259 non-null   object 
 14  CR

In [442]:
test3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9272 entries, 0 to 9271
Data columns (total 48 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ADRES                   9272 non-null   object 
 1   LOT_NO                  9272 non-null   object 
 2   APT_NM                  9262 non-null   object 
 3   EXCL_AREA_SQM           9272 non-null   float64
 4   CONTR_YEAR_MONTH        9272 non-null   int64  
 5   CONTR_DAY               9272 non-null   object 
 6   FLOOR                   9272 non-null   int64  
 7   BUILD_YEAR              9272 non-null   int64  
 8   ADRES_DORO              9272 non-null   object 
 9   CANCEL_REASON_DATE      212 non-null    float64
 10  TRADE_TYPE              9272 non-null   object 
 11  AGNCY_LOCATION          9272 non-null   object 
 12  CODEAPTNM               2690 non-null   object 
 13  HSHLDR_TY               2710 non-null   object 
 14  CRRDPR_TY               2708 non-null   

# 7. Merge school district / 학군

In [443]:
school_district = pd.read_csv('../data/school_district.csv')

In [444]:
school_district.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구       25 non-null     object
 1   학군      25 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 528.0+ bytes


In [445]:
school_district.rename(columns={'구':'gu','학군':'school_district'},inplace=True)

In [446]:
# train2['gu']가 같고, train2['contract_year']가 lease_rate에 있는 연도인 경우 merge
train3 = pd.merge(train3, school_district, how='left', on=['gu'])
test3 = pd.merge(test3, school_district, how='left', on=['gu'])

In [447]:
test3['real_gdp'] = test3['real_gdp'].astype(object)
train3['real_gdp'] = train3['real_gdp'].astype(object)
test3['school_district'] = test3['school_district'].astype(object)
train3['school_district'] = train3['school_district'].astype(object)
test3['LEASE_RATE'] = test3['LEASE_RATE'].astype(object)
train3['LEASE_RATE'] = train3['LEASE_RATE'].astype(object)
test3['interest_rate'] = test3['interest_rate'].astype(object)
train3['interest_rate'] = train3['interest_rate'].astype(object)

# 8. Merge redevelopment / 재개발지역 개수

In [448]:
data1 = pd.read_csv('../data/train_merge_0119_02(cp949).csv',encoding='cp949')
data2 = pd.read_csv('../data/test_merge_0119_02(cp949).csv',encoding='cp949')

In [449]:
train3['redevelop'] = data1['재개발개수']

In [450]:
test3['redevelop'] = data2['재개발개수']

In [451]:
test3['redevelop'] = test3['redevelop'].astype(object)
train3['redevelop'] = train3['redevelop'].astype(object)

# 9. Save train3 & test3

In [452]:
train3.to_csv('../data/train3.csv')
test3.to_csv('../data/test3.csv')

# 10. Change Variables English to Korean 

In [453]:
test3.rename(columns={'ADRES':'시군구','LOT_NO':'번지','APT_NM':'아파트명','EXCL_AREA_SQM':'전용면적',
                      'CONTR_YEAR_MONTH':'계약년월','CONTR_DAY':'계약일','FLOOR':'층','BUILD_YEAR':'건축년도',
                      'ADRES_DORO':'도로명','CANCEL_REASON_DATE':'해제사유발생일',
                      'TRADE_TYPE':'거래유형','AGNCY_LOCATION':'중개사소재지','CODEAPTNM':'k-단지분류',
                      'HSHLDR_TY':'k-세대타입(분양형태)','CRRDPR_TY':'k-복도유형',
                      'HEAT_MTHD':'k-난방방식','ALL_DONG_CO':'k-전체동수','ALL_HSHLD_CO':'k-전체세대수',
                      'CO_WO':'k-건설사','CO_EX':'k-시행사','TOTAR':'k-연면적',
                      'PRIVAREA':'k-주거전용면적','KAPTMPAREA60':'k-전용면적별세대현황60이하',
                      'KAPTMPAREA85':'k-전용면적별세대현황6085이하','KAPTMPAREA135':'k-85135이하',
                      'KAPTMPAREA136':'k-135초과', 'HSHLD_ELCTY_CNTRCT_MTH': '세대전기계약방법',
                      'BU_AR':'건축면적','CNT_PA':'주차대수',
                      'GUBUN':'기타의무임대1234',
                      'X_CODE':'좌표X','Y_CODE':'좌표Y','USE_RQSTDT':'단지신청일',
                      'gu':'구','dong':'동','FULL_ADRES':'전체주소명',
                      'dongAPT_NM':'동아파트명','floating_population':'유동인구',
                      'X_CODE':'좌표X','Y_CODE':'좌표Y','USE_RQSTDT':'단지신청일',
                      'subway_dist':'아파트 지하철역 거리','1stSubArea':'1차역세권',
                      '2ndSubArea':'2차역세권',
                      'bus_dist':'아파트 버스정류장 거리','bridge_dist':'아파트 한강대교 거리',
                      'lease_rate':'전세가율','CONTR':'전체계약일자','interest_rate':'금리',
                      'real_gdp':'실질gdp','nominal_gdp':'명목gdp','school_district':'학군',
                      'redevelop':'재개발개수'
                      },inplace=True)

In [454]:
train3.rename(columns={'ADRES':'시군구','LOT_NO':'번지','APT_NM':'아파트명','EXCL_AREA_SQM':'전용면적',
                      'CONTR_YEAR_MONTH':'계약년월','CONTR_DAY':'계약일','FLOOR':'층','BUILD_YEAR':'건축년도',
                      'ADRES_DORO':'도로명','CANCEL_REASON_DATE':'해제사유발생일',
                      'TRADE_TYPE':'거래유형','AGNCY_LOCATION':'중개사소재지','CODEAPTNM':'k-단지분류',
                      'HSHLDR_TY':'k-세대타입(분양형태)','CRRDPR_TY':'k-복도유형',
                      'HEAT_MTHD':'k-난방방식','ALL_DONG_CO':'k-전체동수','ALL_HSHLD_CO':'k-전체세대수',
                      'CO_WO':'k-건설사','CO_EX':'k-시행사','TOTAR':'k-연면적',
                      'PRIVAREA':'k-주거전용면적','KAPTMPAREA60':'k-전용면적별세대현황60이하',
                      'KAPTMPAREA85':'k-전용면적별세대현황6085이하','KAPTMPAREA135':'k-85135이하',
                      'KAPTMPAREA136':'k-135초과', 'HSHLD_ELCTY_CNTRCT_MTH': '세대전기계약방법',
                      'BU_AR':'건축면적','CNT_PA':'주차대수',
                      'GUBUN':'기타의무임대1234',
                      'X_CODE':'좌표X','Y_CODE':'좌표Y','USE_RQSTDT':'단지신청일',
                      'gu':'구','dong':'동','FULL_ADRES':'전체주소명',
                      'dongAPT_NM':'동아파트명','floating_population':'유동인구',
                      'X_CODE':'좌표X','Y_CODE':'좌표Y','USE_RQSTDT':'단지신청일',
                      'subway_dist':'아파트 지하철역 거리','1stSubArea':'1차역세권',
                      '2ndSubArea':'2차역세권',
                      'bus_dist':'아파트 버스정류장 거리','bridge_dist':'아파트 한강대교 거리',
                      'lease_rate':'전세가율','CONTR':'전체계약일자','interest_rate':'금리',
                      'real_gdp':'실질gdp','nominal_gdp':'명목gdp','school_district':'학군',
                      'redevelop':'재개발개수'
                      },inplace=True)

# 11. Save Korean train3 & test3 

In [455]:
train3.to_csv('../data/korean_train3.csv')
test3.to_csv('../data/korean_test3.csv')